In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.10795474052429199
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 16.355843151667198
200000 27.482428370290304
300000 30.77161513864589
400000 30.648754715074826
500000 31.578810753245968
600000 32.158183530429376
700000 31.036561944889637
800000 30.69150903747061
900000 30.42351483928149
1000000 31.277812181038176
1100000 13.193545845346115
1200000 27.624474689690228
1300000 30.81845920764445
1400000 30.11359199267247
1500000 31.125959132511305
1600000 30.984660194748677
1700000 32.15509674481135
1800000 30.52815075812511
1900000 30.368981858400403
2000000 31.547322634401933
2100000 30.865011774901813
2200000 8.183635655404842
2300000 34.61727198613394
2400000 36.128499965929926
2500000 36.172552662064575
2600000 35.58691112525904
2700000 34.9407069797094
2800000 34.569110932677255
2900000 32.811339449463276
3000000 34.

29500000 29.133990086753144
29600000 30.612559728677468
29700000 29.133070468806334
29800000 27.885542743104086
29900000 16.094510532483795
30000000 28.66700917353155
30100000 32.73968068743221
30200000 29.567084402771698
30300000 20.27394464091877
30400000 23.15485196958701
30500000 29.81577751651082
30600000 31.301630307748137
30700000 29.29927469211957
30800000 30.44228093282418
30900000 28.38130973714319
31000000 27.74036941698329
31100000 25.791493358778272
31200000 13.621801678760772
31300000 27.49717708696289
31400000 27.70576576092546
31500000 28.035476728504687
31600000 30.639661698344593
31700000 28.207442421942513
31800000 30.517987025787438
31900000 28.36453959658895
32000000 13.52856830348084
32100000 32.217635442095705
32200000 29.044805247007584
32300000 31.56997274788757
32400000 29.392605622477745
32500000 12.252255918666025
32600000 32.78356209098211
32700000 28.35262210899894
32800000 30.2222490428231
32900000 28.303907620835272
33000000 27.365025017971497
33100000 2

59200000 28.079984317769142
59300000 24.696974049538458
59400000 26.003554170130467
59500000 27.047918252489552
59600000 27.18812475110879
59700000 14.330593655928622
59800000 27.560843118321216
59900000 31.183381678416556
60000000 29.267701223877374
60100000 30.478598061690214
60200000 28.727831675285593
60300000 29.559411157673694
60400000 29.390224474200583
60500000 28.322378965335727
60600000 16.78655699440678
60700000 22.056165333818264
60800000 27.145295801623455
60900000 25.841962854533406
61000000 25.13395290623015
61100000 26.271387536355114
61200000 12.972719078778423
61300000 21.88596862588696
61400000 27.15724334929769
61500000 24.58221982999895
61600000 26.348039218887823
61700000 25.47468143695094
61800000 14.930139215580827
61900000 25.38317407354141
62000000 27.816685200437348
62100000 29.17539078848408
62200000 28.372892095253125
62300000 29.507250990332835
62400000 28.588317325058338
62500000 29.14928950714038
62600000 27.593271792119406
62700000 29.37278671939838
628

88900000 23.79317803056155
89000000 21.73790844091032
89100000 12.47208919432749
89200000 19.63924371104361
89300000 21.575101093100677
89400000 21.900402768488945
89500000 13.17540713525439
89600000 20.295590283704094
89700000 22.233966871354873
89800000 24.426470243179327
89900000 22.754030941307075
90000000 26.670369082860116
90100000 23.567337153615505
90200000 26.706698878702646
90300000 23.841754066317865
90400000 25.42229529909433
90500000 25.651083942558486
90600000 23.025750877888534
90700000 25.00243529188838
90800000 21.244663899779244
90900000 11.145006338830205
91000000 23.138758658911385
91100000 22.051696444104188
91200000 23.69363814886807
91300000 9.1656071762054
91400000 21.548949459065124
91500000 24.483880953334122
91600000 15.768875110928688
91700000 16.951337046312663
91800000 23.422781676759172
91900000 21.83586032646529
92000000 24.666521259870752
92100000 23.400824330208014
92200000 25.31943570837594
92300000 25.07751803243766
92400000 23.534160366408514
925000

118000000 23.79356490023213
118100000 23.382234162609983
118200000 22.09952025390645
118300000 24.85228529545297
118400000 19.75482383856165
118500000 22.677216141871043
118600000 20.102334406211906
118700000 20.758390734115146
118800000 22.185632595476648
118900000 19.624354717049442
119000000 15.072799022004004
119100000 18.060360034027624
119200000 7.0685438974875945
119300000 14.250924809565278
119400000 17.496119458596702
119500000 20.01639729461317
119600000 22.249674863957498
119700000 20.20044843475263
119800000 22.311852756541228
119900000 21.755024766359632
120000000 21.4816786053724
120100000 23.75521997358716
120200000 22.462586814456586
120300000 25.631296783090086
120400000 21.90319359597409
120500000 22.97786966841053
120600000 21.410992053075216
120700000 20.7682042267632
120800000 23.550663761610746
120900000 19.904396524936928
121000000 22.92411725561493
121100000 21.015714706341814
121200000 8.506614268366986
121300000 15.159631205250527
121400000 14.12591925791121
1

...

289900000 5.925535212247364
290000000 5.111826768304067
290100000 5.4666173231187605
290200000 4.632669787893543
290300000 5.27760506259713
290400000 5.040724330170523
290500000 5.3310671163259595
290600000 5.216629611007619
290700000 4.737408051849514
290800000 3.617843506198023
290900000 1.1363063154983997
291000000 1.1427366805601518
291100000 5.818688497341928
291200000 4.788146836900805
291300000 5.212791164321484
291400000 5.16890643515387
291500000 4.930095126989406
291600000 5.008808580056537
291700000 4.832424901613334
291800000 5.282863626037337
291900000 4.972314354605396
292000000 6.215408977533279
292100000 5.396125166353593
292200000 5.257232447841586
292300000 5.184671499669291
292400000 4.72431260441354
292500000 5.242998266153468
292600000 4.946587053773244
292700000 5.206437608236653
292800000 4.715991384901732
292900000 5.307783508334919
293000000 1.1253947796490371
293100000 1.1296130685823262
293200000 3.9765413615272904
293300000 4.738368334201311
293400000 4.9453

319000000 3.5559138735544082
319100000 3.762306740075489
319200000 3.6355735254673625
319300000 3.860200462702327
319400000 4.085421360332733
319500000 3.7337645019296666
319600000 3.685873789990498
319700000 3.515267417153325
319800000 2.96123810387498
319900000 4.701213152358669
320000000 4.584086675617029
320100000 4.006852906521932
320200000 3.798118733517464
320300000 3.693164127557963
320400000 3.700097502177119
320500000 3.9507157917329527
320600000 3.381988651729296
320700000 3.4930534237992337
320800000 3.5976057079602937
320900000 1.0854684381921669
321000000 0.8163515072966391
321100000 1.992261941921156
321200000 3.7470619816481947
321300000 2.1902464066719487
321400000 3.369656171679266
321500000 3.743537342469829
321600000 3.548733398634975
321700000 2.9675554358796146
321800000 3.575914068187209
321900000 4.325764951423794
322000000 3.6705804931439796
322100000 5.170345321039712
322200000 4.202977877516915
322300000 4.627449009119132
322400000 3.9325985531591092
32250000

347700000 1.419055593804394
347800000 1.752356974006698
347900000 1.4855245052379074
348000000 1.5018284520804959
348100000 1.384670264284309
348200000 1.2952370872080923
348300000 1.2818332177873153
348400000 1.2272166262025879
348500000 1.2965986258749407
348600000 1.2880020292470575
348700000 1.37058707336984
348800000 0.5064152177842319
348900000 0.36939272521425426
349000000 1.4560675276310326
349100000 1.3810387071413517
349200000 1.3138974610891911
349300000 1.3188588478428644
349400000 2.1917563245997767
349500000 2.2808282039724292
349600000 1.9312576053093256
349700000 1.3896421642006895
349800000 1.3439409911344349
349900000 1.4226232007192188
350000000 1.504511265045476
350100000 1.457814406523812
350200000 1.3815100682884667
350300000 1.2966637112819472
350400000 1.2166204768613214
350500000 1.8364801628982381
350600000 1.300924439763226
350700000 1.295029021558622
350800000 1.2184244820803258
350900000 1.0042827857297003
351000000 0.3237637323103547
351100000 0.3352862868

375700000 0.7462314775405486
375800000 0.15992762666253843
375900000 0.1668389444839292
376000000 0.1615366732236319
376100000 0.16367742776520253
376200000 0.16405559663954444
376300000 0.1797551699847963
376400000 0.16843632686904272
376500000 0.2186875431259804
376600000 0.2908683406010714
376700000 0.16281752300336363
376800000 0.23789741961421237
376900000 0.2583775114525629
377000000 0.16717305963909032
377100000 0.2671952102020813
377200000 0.2856736754274792
377300000 0.2847753831450045
377400000 0.1671017917632593
377500000 0.1666888471937365
377600000 0.16892233424817918
377700000 0.17005941558631196
377800000 0.169558575709737
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 0.9383517801087498
378000000 1.409903653220578
378100000 1.0077873467417888
378200000 1.9866430952994067
378300000 0.937519136498428
378400000 1.3552437899358736
378500000 1.363864016065864
378600000 1.162955655148965
378700000 0.9334919230930235
378800000 0.90691297626927

403800000 0.7805244135963837
403900000 0.6871523237535934
404000000 0.8383520393191337
404100000 0.6740309938496178
404200000 0.7914020433248314
404300000 0.7641742302402635
404400000 0.7856258401870614
404500000 0.7625784003049865
404600000 0.679470919446806
404700000 0.8575647732181932
404800000 0.690712897471847
404900000 0.7364261168667078
405000000 0.7825691406362878
405100000 0.7896214339098057
405200000 0.7483449164676785
405300000 0.6544725967774452
405400000 0.8225582607528137
405500000 0.697084006427302
405600000 0.6985661230461538
405700000 0.7850167831299483
405800000 0.7485314497169971
405900000 0.7374367871253358
406000000 0.6529197563766744
406100000 0.7608131363349377
406200000 0.7215909652568698
406300000 0.6689606503633612
406400000 0.7343706458055078
406500000 0.7380020465972337
406600000 0.6538936192694916
406700000 0.6060458500885785
406800000 0.6679018650137616
406900000 0.6939080916526722
407000000 0.6208353254632394
407100000 0.6283489062822607
407200000 0.81076